### Merging
##### Join multiple dataframes together

In [1]:
import pandas as pd
import sqlite3

In [2]:
emp_db = sqlite3.connect("../../data/company_employees_plus.db")

employees = pd.read_sql("SELECT * FROM employees", con = emp_db)
countries = pd.read_sql("SELECT * FROM country_master", con = emp_db)
continents = pd.read_sql("SELECT * FROM continent_master", con = emp_db)

print(len(employees), len(countries), len(continents))

928 6 4


In [3]:
countries.head()

,country,continent,population
0,Mexico,North America,146000000
1,Canada,North America,32000000
2,Ethiopia,Africa,17000000
3,Kenya,Africa,37000000
4,Congo,Africa,22000000


In [4]:
continents.head()

,name,population
0,North America,579000000
1,South America,422500000
2,Asia,4436000000
3,Africa,1216000000


In [5]:
employees.merge(right = countries, on = "country", how = "inner").head()

,id,first_name,last_name,email,gender,company,total_worth,is_management,country,continent,population
0,02-8563405,Lorri,Laurenson,llaurenson0@symantec.com,Female,Yakijo,35.0515,true,Mexico,North America,146000000
1,50-7875167,Alexandre,Oxbrough,aoxbrough2@google.ca,Male,Chatterpoint,40.7939,true,Mexico,North America,146000000
2,55-6753846,Hendrika,Sollett,hsollett3@whitehouse.gov,Female,Vimbo,36.7514,false,Mexico,North America,146000000
3,84-7082301,Corrianne,Rodge,crodge4@topsy.com,Female,Agivu,9.9857,false,Mexico,North America,146000000
4,55-8417539,Micheil,Boullin,mboullin8@google.cn,Male,Brainverse,16.5639,true,Mexico,North America,146000000


In [6]:
# If the names of the columns you want to join on have different names, then you have to make them the same

(countries
    .merge(
        right = continents.assign(continent = continents.name), 
        on = "continent", 
        how = "inner"
    )
)

,country,continent,population_x,name,population_y
0,Mexico,North America,146000000,North America,579000000
1,Canada,North America,32000000,North America,579000000
2,Ethiopia,Africa,17000000,Africa,1216000000
3,Kenya,Africa,37000000,Africa,1216000000
4,Congo,Africa,22000000,Africa,1216000000
5,Algeria,Africa,8000000,Africa,1216000000


In [7]:
# There is a rename method that might make our lives a little easier, prevent duplicate column names

(countries
    .merge(
        right = continents.rename(columns = {"name": "continent"}), 
        on = "continent", 
        how = "inner"
    )
)
# Note that we end up with two columns named 'population' that we have to handle

,country,continent,population_x,population_y
0,Mexico,North America,146000000,579000000
1,Canada,North America,32000000,579000000
2,Ethiopia,Africa,17000000,1216000000
3,Kenya,Africa,37000000,1216000000
4,Congo,Africa,22000000,1216000000
5,Algeria,Africa,8000000,1216000000


In [8]:
# We can use strategically placed renames to make this smooth as well.
(countries
    .rename(columns = {"population": "pop_country"})
    .merge(
        right = continents.rename(columns = {"name": "continent", "population": "pop_continent"}), 
        on = "continent", 
        how = "inner"
    )
)

,country,continent,pop_country,pop_continent
0,Mexico,North America,146000000,579000000
1,Canada,North America,32000000,579000000
2,Ethiopia,Africa,17000000,1216000000
3,Kenya,Africa,37000000,1216000000
4,Congo,Africa,22000000,1216000000
5,Algeria,Africa,8000000,1216000000


In [9]:
# We can separate the left/right join column names, but will keep both in the output.
(countries
    .merge(
        right = continents, 
        left_on = "continent",
        right_on = "name",
        how = "inner"
    )
)

,country,continent,population_x,name,population_y
0,Mexico,North America,146000000,North America,579000000
1,Canada,North America,32000000,North America,579000000
2,Ethiopia,Africa,17000000,Africa,1216000000
3,Kenya,Africa,37000000,Africa,1216000000
4,Congo,Africa,22000000,Africa,1216000000
5,Algeria,Africa,8000000,Africa,1216000000


In [10]:
# These patterns work for other types of joins/merges
(countries
    .rename(columns = {"population": "pop_country"})
    .merge(
        right = continents.rename(columns = {"name": "continent", "population": "pop_continent"}), 
        on = "continent", 
        how = "right"
    )
)

,country,continent,pop_country,pop_continent
0,Mexico,North America,146000000.0,579000000
1,Canada,North America,32000000.0,579000000
2,Ethiopia,Africa,17000000.0,1216000000
3,Kenya,Africa,37000000.0,1216000000
4,Congo,Africa,22000000.0,1216000000
5,Algeria,Africa,8000000.0,1216000000
6,NaN,South America,NaN,422500000
7,NaN,Asia,NaN,4436000000


### Merging with an Aggregate

In [11]:
agg_comp = (employees
    [["company", "total_worth"]]
    .groupby("company").mean()
    .rename(columns = {"total_worth":"avg_worth"})
)

merged = (employees
    [["id", "first_name", "last_name", "company", "total_worth"]]
    .merge(
        right = agg_comp[agg_comp.avg_worth > 40],
        on = "company",
        how = "inner"
    )
)
merged[merged.total_worth > merged.avg_worth]

,id,first_name,last_name,company,total_worth,avg_worth
1,31-3382933,Gale,Mordey,Yakijo,53.3896,41.753000
11,02-7129981,Verney,Hounson,Quimba,48.7559,42.293100
13,73-2989962,Maynard,Crich,Quaxo,54.9417,43.868750
16,37-4504415,Grace,Durtnel,Zoonder,48.1180,43.838433
23,84-6496630,Seth,Clough,Shuffletag,51.7544,41.393900
25,53-0802660,Misha,Jolliff,Dabshots,45.6971,43.682933
27,34-7663720,Leonard,Gateland,Dabshots,61.0984,43.682933
32,59-4074538,Tiffanie,Jemmison,Realbridge,62.4443,55.610700
